#### Continuous p-Fuzzy System: Predator-Prey Model

Models ecological predator-prey dynamics using continuous p-fuzzy differential equations where population change rates are determined by fuzzy inference rules instead of fixed parameters.

## Installation and Imports

In [ ]:
# Install pyfuzzy-toolbox for fuzzy systems modeling
!pip install pyfuzzy-toolbox matplotlib numpy -q

In [ ]:
import sys
import os

import fuzzy_systems as fs
from fuzzy_systems.dynamics import PFuzzyContinuous
from fuzzy_systems.inference import MamdaniSystem
import numpy as np
import matplotlib.pyplot as plt

print("✅ Libraries imported successfully!")

In [ ]:
# Create Mamdani system for predator-prey dynamics
# Inputs: current prey and predator populations
# Outputs: change rates (dx/dt, dy/dt)

fis = MamdaniSystem(name="Continuous Predator-Prey")

# ========================================
# INPUTS: Prey (x) and Predators (y)
# ========================================

# Define universe [0, 100] for both populations
fis.add_input('prey', (0, 100))
fis.add_input('predator', (0, 100))

# Linguistic terms: B (Low), MB (Medium-Low), MA (Medium-High), A (High)
for var in ['prey', 'predator']:
    fis.add_term(var, 'B', 'gaussian', (0, 12))
    fis.add_term(var, 'MB', 'gaussian', (100/3, 12))
    fis.add_term(var, 'MA', 'gaussian', (2*100/3, 12))
    fis.add_term(var, 'A', 'gaussian', (100, 12))

fis.plot_variables()

In [ ]:
# ========================================
# OUTPUTS: Population change rates (dx/dt, dy/dt)
# ========================================

# Output 1: Δprey (prey growth/decline rate)
# Negative terms (_n) for decline, positive (_p) for growth

var_prey_universe = (-2, 2)
lrg = 0.5

fis.add_output('var_prey', var_prey_universe)
fis.add_term('var_prey', 'B_n', 'triangular', (-lrg, 0, 0))
fis.add_term('var_prey', 'MB_n', 'triangular', (-2*lrg, -lrg, 0))
fis.add_term('var_prey', 'MA_n', 'triangular', (-3*lrg, -2*lrg, -lrg))
fis.add_term('var_prey', 'A_n', 'trapezoidal', (-4*lrg, -4*lrg, -3*lrg, -2*lrg))
fis.add_term('var_prey', 'B_p', 'triangular', (0, 0, lrg))
fis.add_term('var_prey', 'MB_p', 'triangular', (0, lrg, 2*lrg))
fis.add_term('var_prey', 'MA_p', 'triangular', (lrg, 2*lrg, 3*lrg))
fis.add_term('var_prey', 'A_p', 'trapezoidal', (2*lrg, 3*lrg, 4*lrg, 4*lrg))

# Output 2: Δpredator (predator growth/decline rate)

var_predator_universe = (-2, 2)
lrg = 0.5

fis.add_output('var_predator', var_predator_universe)
fis.add_term('var_predator', 'B_n', 'triangular', (-lrg, 0, 0))
fis.add_term('var_predator', 'MB_n', 'triangular', (-2*lrg, -lrg, 0))
fis.add_term('var_predator', 'MA_n', 'triangular', (-3*lrg, -2*lrg, -lrg))
fis.add_term('var_predator', 'A_n', 'trapezoidal', (-4*lrg, -4*lrg, -3*lrg, -2*lrg))
fis.add_term('var_predator', 'B_p', 'triangular', (0, 0, lrg))
fis.add_term('var_predator', 'MB_p', 'triangular', (0, lrg, 2*lrg))
fis.add_term('var_predator', 'MA_p', 'triangular', (lrg, 2*lrg, 3*lrg))
fis.add_term('var_predator', 'A_p', 'trapezoidal', (2*lrg, 3*lrg, 4*lrg, 4*lrg))

fis.plot_variables(['var_prey', 'var_predator'])


In [ ]:
# ========================================
# FUZZY RULE BASE (16 rules)
# ========================================

# Define expert rules for predator-prey interaction
# Format: ('prey_term', 'predator_term', 'd_prey_term', 'd_predator_term')
# Logic: Low predators + prey → prey grows; High predators → prey declines

rules = [
    # Row 1: prey = B (low)
    ('B', 'B', 'MB_p', 'MB_n'),
    ('B', 'MB', 'B_p', 'MB_n'),
    ('B', 'MA', 'B_n', 'MA_n'),
    ('B', 'A', 'MB_n', 'A_n'),
    
    # Row 2: prey = MB (medium-low)
    ('MB', 'B', 'MA_p', 'B_n'),
    ('MB', 'MB', 'MB_p', 'B_n'),
    ('MB', 'MA', 'B_n', 'MB_n'),
    ('MB', 'A', 'MB_n', 'MA_n'),
    
    # Row 3: prey = MA (medium-high)
    ('MA', 'B', 'MB_p', 'MA_p'),
    ('MA', 'MB', 'B_p', 'MB_p'),
    ('MA', 'MA', 'MB_n', 'B_p'),
    ('MA', 'A', 'MA_n', 'B_p'),
    
    # Row 4: prey = A (high)
    ('A', 'B', 'B_n', 'A_p'),
    ('A', 'MB', 'MB_n', 'MA_p'),
    ('A', 'MA', 'MA_n', 'MB_p'),
    ('A', 'A', 'A_n', 'B_p')
]

# Add rules and visualize the rule matrix
fis.add_rules(rules)
fis.plot_rule_matrix()

# Optional: Use fis.save('filename.json') to save the complete FIS

In [ ]:
# Create continuous p-fuzzy dynamical system
# Mode 'absolute': dx/dt = f(x) where f is the fuzzy inference
# Method 'euler': numerical integration using Euler's method

pfuzzy = PFuzzyContinuous(
    fis=fis,
    mode='absolute',
    state_vars=['prey', 'predator'],
    method='euler'
)


In [ ]:
# Simulate the system with adaptive step size
# Initial: 40 prey, 20 predators | Time span: [0, 250]
# Adaptive integration improves accuracy by adjusting step size

t, trajectory = pfuzzy.simulate(
    x0={'prey': 40, 'predator': 20},
    t_span=(0, 250),
    adaptive=True,
    tolerance=1e-4,
)


In [ ]:
# Plot population dynamics over time
# Shows cyclic oscillations typical of predator-prey systems

fig, ax = pfuzzy.plot_trajectory(
    variables=['prey', 'predator'],
    figsize=(14, 6),
    title='Continuous Fuzzy Lotka-Volterra System',
    xlabel='Time',
    ylabel='Population'
)

plt.show()

In [ ]:
# Plot phase space trajectory (prey vs predator)
# Circular/elliptical orbits indicate stable limit cycles

fig, ax = pfuzzy.plot_phase_space(
    'prey', 'predator',
    figsize=(8, 5),
    title='Phase Space: Predator-Prey Cycle'
)

plt.show()

In [ ]:
# Create vector field showing system dynamics at each point
# Arrows indicate direction and magnitude of population change

prey_grid = np.linspace(10, 90, 12)
pred_grid = np.linspace(10, 90, 12)
P, D = np.meshgrid(prey_grid, pred_grid)

# Calculate field vectors (dx/dt, dy/dt) at each grid point
dP = np.zeros_like(P)
dD = np.zeros_like(D)

for i in range(len(prey_grid)):
    for j in range(len(pred_grid)):
        state = {'prey': P[j, i], 'predator': D[j, i]}
        result = fis.evaluate(state)
        dP[j, i] = result['var_prey']
        dD[j, i] = result['var_predator']

# Plot vector field with trajectory overlay
fig, ax = plt.subplots(figsize=(12, 10))

# Vector field shows system dynamics at each point
ax.quiver(P, D, dP, dD, alpha=0.5, color='gray', width=0.003)

# Overlay the simulated trajectory
ax.plot(trajectory[:, 0], trajectory[:, 1], 'b-', linewidth=2.5, 
        label='Trajectory', alpha=0.8)
ax.plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=12, 
        label='Initial', zorder=5)
ax.plot(trajectory[-1, 0], trajectory[-1, 1], 'ro', markersize=12, 
        label='Final', zorder=5)

ax.set_xlabel('Prey', fontsize=13)
ax.set_ylabel('Predators', fontsize=13)
ax.set_title('Vector Field + Trajectory of p-Fuzzy System', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)

plt.tight_layout()
plt.show()


In [ ]:
# Compare multiple initial conditions
# Shows that different starting points lead to different limit cycles
initial_conditions = [
    {'prey': 30, 'predator': 15},
    {'prey': 50, 'predator': 25},
    {'prey': 70, 'predator': 35},
    {'prey': 40, 'predator': 40}
]

colors = ['blue', 'red', 'green', 'purple']

# Create figure with 2 subplots (temporal and phase space)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

print("Simulating multiple initial conditions...\n")

for i, x0 in enumerate(initial_conditions):
    print(f"{i+1}. Prey={x0['prey']}, Predators={x0['predator']}")
    
    # Simulate from this initial condition
    t, traj = pfuzzy.simulate(x0=x0, t_span=t_span)
    
    # Temporal plot (dashed=prey, solid=predator)
    ax1.plot(t, traj[:, 0], '--', color=colors[i], alpha=0.6, linewidth=1.5)
    ax1.plot(t, traj[:, 1], '-', color=colors[i], linewidth=2, 
             label=f"IC{i+1}: ({x0['prey']}, {x0['predator']})")
    
    # Phase space plot
    ax2.plot(traj[:, 0], traj[:, 1], color=colors[i], linewidth=2.5, alpha=0.7)
    ax2.plot(traj[0, 0], traj[0, 1], 'o', color=colors[i], markersize=10)

# Configure temporal plot
ax1.set_xlabel('Time', fontsize=12)
ax1.set_ylabel('Population', fontsize=12)
ax1.set_title('Temporal Dynamics\n(solid line=predator, dashed=prey)', 
              fontsize=12, fontweight='bold')
ax1.legend(loc='best', fontsize=9)
ax1.grid(True, alpha=0.3)

# Configure phase space plot
ax2.set_xlabel('Prey', fontsize=12)
ax2.set_ylabel('Predators', fontsize=12)
ax2.set_title('Phase-space\n(circle=initial condition)', 
              fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 100)
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()


In [ ]:
# Export simulation results to CSV for further analysis
pfuzzy.to_csv('/tmp/predator_prey_continuous.csv')

print("✅ Data exported to: /tmp/predator_prey_continuous.csv")
print("\n📄 First 10 rows:")

import pandas as pd
df = pd.read_csv('/tmp/predator_prey_continuous.csv')
print(df.head(10))